<a href="https://colab.research.google.com/github/vinayk19/Assignment/blob/master/word_and_countryfinding_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd /content/drive/My Drive/AI/basic/data/names_train

/content/drive/My Drive/AI/basic/data/names_train


In [3]:
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np


# from name_dataset import NameDataset
# from torch.nn.utils.rnn import pack_padded_sequence, pad_paked_sequence

In [7]:
xy1 = pd.read_csv("names_train.csv") #q is it require to convert it(text) into torch
# xy1.shape
# name = (xy1.iloc[0,:][0])
# name
xy1.describe


<bound method NDFrame.describe of               Adsit   Czech
0            Ajdrna   Czech
1      Antonowitsch   Czech
2        Antonowitz   Czech
3        Ballalatak   Czech
4        Ballaltick   Czech
...             ...     ...
13368         Zabek  Polish
13369     Zdunowski  Polish
13370     Zdunowski  Polish
13371      Ziemniak  Polish
13372       Zientek  Polish

[13373 rows x 2 columns]>

In [10]:
word = xy1['Adsit'].tolist()
country = xy1['Czech'].tolist()
print(len(word))
print(len(country))
print(list(dict.fromkeys(country))) # it will provide only unique country name only

13373
13373
['Czech', 'German', 'Arabic', 'Japanese', 'Chinese', 'Vietnamese', 'Russian', 'French', 'Irish', 'English', 'Spanish', 'Greek', 'Italian', 'Portuguese', 'Scottish', 'Dutch', 'Korean', 'Polish']


Tast1 : create database

In [27]:
class Name(Dataset):
  def __init__(self, csv_file):
  # def __init__(self, csv_file):
    self.xy = pd.read_csv(csv_file)
    self.word = self.xy['Adsit'].tolist()
    self.country = self.xy['Czech'].tolist()
    self.country_list = list(dict.fromkeys(self.country))
    
  def __getitem__(self, id):
    return self.word[id], self.country[id]

  def __len__(self):
    return len(self.word)

  def get_country_list(self):
    return self.country_list
  #get the unique country name by its id or lebel
  def get_country_by_id(self, id):
    return self.country_list[id] #@learning callable meance function() but in list it is [] (as list is not callable)
  
  def get_id_by_country(self, country):
    return self.country_list.index(country)

In [25]:
#test database
if __name__ == '__main__':
  database = Name('/content/drive/My Drive/AI/basic/data/names_train/names_train.csv')
  print(database.get_country_list())
  print(database.get_id_by_country('French'))
  print(database.get_country_by_id(7))

['Czech', 'German', 'Arabic', 'Japanese', 'Chinese', 'Vietnamese', 'Russian', 'French', 'Irish', 'English', 'Spanish', 'Greek', 'Italian', 'Portuguese', 'Scottish', 'Dutch', 'Korean', 'Polish']
7
French


tast2: Create dataloader

In [50]:
# train loader
BS = 10
# database = 
database = Name('/content/drive/My Drive/AI/basic/data/names_train/names_train.csv')
train_loader = DataLoader(dataset=database, batch_size= BS, shuffle=True)
print(len(database))
print(database[2000])

13373
('Samaha', 'Arabic')


In [55]:
#train loader
if __name__=='__main__':
  database = Name('/content/drive/My Drive/AI/basic/data/names_train/names_train.csv')
  # database = Name('names_test.csv')
  print(len(database))
  print(len(database[5]))

13373
2


In [ ]:
#test loader
database_test = Name('/content/drive/My Drive/AI/basic/data/names_train/names_test.csv')
test_loader = DataLoader(dataset=database_test, batch_size= BS, shuffle=True)

working on asc and tranposzing .t() is it tup[le , tensor or what then next action
1. bs is 10
2. test funciton
validation


task3: model: embedding, rnn, linear layer 
task4: optimizer
task5: training
taask6: test

In [107]:
class net(torch.nn.Module): #learning we can get fuction created inside database (it can be train or test)
  def __init__(self):
    super(net,self).__init__()
    self.word_len = 18 # seq chenged from 15 to 18 for test
    self.vocab_size = self.word_len # no of character per word
    self.embedding_dim = 6 # how many diamention it wants
    self.input_size = self.embedding_dim #6
    self.hiddenO = self.input_size #6 dim
    self.hidden_size = self.embedding_dim #6
    self.batch = 1
    self.num_layer =1
    self.num_class = 18 #num of contries

    #embedding  @learning use tensor conversion al last meand at GRU ot RNN time not beofore that as arugment of other function (like embed) are not tensors
    self.embed = nn.Embedding(num_embeddings = self.vocab_size, embedding_dim = torch.LongTensor([self.embedding_dim])) # num_enn = input size word lenth and embedd is output size ie hidden size
    #gru    
    self.grucell = nn.GRU(input_size = self.input_size, hidden_size = self.hidden_size, num_layers =self.num_layer, batch_first = True)
    #fc
    self.fc = nn.Linear(in_features= self.embedding_dim, out_features= self.num_class)
  
  def forward(self, word, country): # 10 word and 10 country
    # word, contry = data
    print("word", word) # word type is tuple of B*S
    #converting work in to integer ASCII for input and outpuut ord # learning asc is not required as embedding can take words directly
     
    word_ascii, w_len = self.asc(word, self.word_len) #in: BS * word and out: BS * ASCII with padding
    print("\n word_ascii", word_ascii,  "\n w_len", w_len)
    # challenge we need some padding. keep every word as 18 character long.
    #learning embedding needs word(s) * BS  ir S*B not B*S
    word_ascii = word_ascii.t() # in B*S ot S*B
    word_embed = self.embed(word_ascii) #embedding e extra in matrix out : BS* word_len*E
    #rnn in and out rnn_in BS*Seq*inu_D(e) and rnn_out BS*Seq*hiddenO
    # input = B*Seq_len*input, hidden = num_lauyer* batch,hidden_size
    input = word_embed # B*S*I
    hidden = self.hidden_init()
    out, h_n = self.grucell(input, hidden) #out B*S*hidden [1 10 6] , h_n = num_layerxBxHidden [1 1 6]

    #FC linear layer in Seq to out no of classor contry(-1, nod of class or country)
    out = self.fc(h_n) #in [1 1 6] out [1 1 18]
    out = out.view(-1, self.num_class)
    # out - out.t()
    country_ids = self.country2tensor(country) # 18 contoury with each 15 word
    # country = country.t()#view(-1)
    # print("country ids", country_ids.shape)
    # country = country.squeeze_()
    # out = out.squeeze_()
    return out, country_ids

    #output output of DC 
  def hidden_init(self):
    hid = torch.zeros(self.num_layer, self.batch, self.hiddenO)
    return hid
  def country2tensor(self, countries):
    # print("contries size", len(countries))
    country_ids = [database.get_id_by_country(each_country) for each_country in countries]
    # print("contries ids", len(country_ids))
    return torch.LongTensor(country_ids)

  def asc(self, words, word_len):
    word_len = word_len
    sorks3 =  []
    # sorks3 = torch.zeros[len(words)]
    for word in words:
      # # for cha in word:
      # sorks = [ord(cha) for cha in word]
      # sorks2 = [sorks + [0]*(word_len-len(sorks))] 
      # sorks3 = Variable(torch.LongTensor([sorks2]))
      # sorks4 = sorks4.append(sorks3)
      
      # sorks3.append(Variable(torch.LongTensor([[ord(cha) for cha in word] + [0]*(word_len-len([ord(cha) for cha in word]))])))
      sorks3.append([[ord(cha) for cha in word] + [0]*(word_len-len([ord(cha) for cha in word]))])
      # i = i+1
    return sorks3, len(sorks3) 


model = net()

Process contry differently as each contry will have unique no (one number not as matrix as word (N * no of class) will be comparing or predicting the perticular class. so its a one idea to keep perticular class as one number rather than keeping it as one class as array.

TODO each contry as one unique no..

In [88]:
Criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = .0001)


Trainig

https://stackoverflow.com/questions/49206550/pytorch-error-multi-target-not-supported-in-crossentropyloss

In [108]:
for epoch in range(1):
  total_loss = 0 
  # batch_id = 0 
  for batch_idx, (word, country) in enumerate(train_loader):
    print("word database", len(word), "countey database", len(country))
    out, country = model(word, contry)
    print("out shape", out.shape, "countey shape", country.shape)
    optimizer.zero_grad()
    loss = Criterion(out, country)
    loss.backward()
    optimizer.step()
    if batch_idx % 500  == 0:
        print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(word), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    

word database 10 countey database 10
word ('Kubo', 'Andryuhin', 'Teoh', 'Ville', 'Bishara', 'Baiguzov', 'Rennalls', 'Bairamkuloff', 'Neary', 'Turmanov')

 word_ascii [[[75, 117, 98, 111, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[65, 110, 100, 114, 121, 117, 104, 105, 110, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[84, 101, 111, 104, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[86, 105, 108, 108, 101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[66, 105, 115, 104, 97, 114, 97, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[66, 97, 105, 103, 117, 122, 111, 118, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[82, 101, 110, 110, 97, 108, 108, 115, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[66, 97, 105, 114, 97, 109, 107, 117, 108, 111, 102, 102, 0, 0, 0, 0, 0, 0]], [[78, 101, 97, 114, 121, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[84, 117, 114, 109, 97, 110, 111, 118, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]] 
 w_len 10


AttributeError: ignored

In [ ]:
n= ["name"]
# n[1]
# ord(n[1])
print(asc(n,10)[0].size())
a, b = asc(n,10)
embed=nn.Embedding(200,3)
emb2 = embed(a)



In [ ]:
print(emb2.shape)

In [ ]:
  sorks= []
  i=0
  def asc(words, word_len):
    word_len = word_len

    for word in words:
      # for cha in word:
      sorks = [ord(cha) for cha in word]
      sorks2 = sorks + [0]*(word_len-len(sorks)) 
      sorks3 = Variable(torch.LongTensor([sorks2]))
    return sorks3, len(sorks2)

In [ ]:
n[1]

In [ ]:
n1 =asc(n)

In [ ]:
n2 = concat(n1, zeros[2])

In [ ]:
like_zero[2]